# IMPORTS

In [15]:
import ccxt
import pandas as pd
import time
from datetime import datetime
# from config import myconfig



# INITIALIZE

In [16]:
"ftx" in ccxt.exchanges

True

In [17]:
exchange = ccxt.exmo({
})

exchange

ccxt.exmo()

In [18]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets]
print(f'No. of market symbols: {len(market_symbols)}')
print(f'Sample:{market_symbols[0:5]}')

No. of market symbols: 183
Sample:['BTC/USDT', 'BTC/USD', 'BTC/EUR', 'BTC/GBP', 'BTC/UAH']


# STEP 1: GET ALL THE CRYPTO COMBINATIONS FOR USDT

In [19]:
def get_crypto_combinations(market_symbols, base):
    combinations = []
    for sym1 in market_symbols:
        
        sym1_token1 = sym1.split('/')[0]
        sym1_token2 = sym1.split('/')[1]
        
        if (sym1_token2 == base):
            for sym2 in market_symbols:
                sym2_token1 = sym2.split('/')[0]
                sym2_token2 = sym2.split('/')[1]
                if (sym1_token1 == sym2_token2):
                    for sym3 in market_symbols:
                        sym3_token1 = sym3.split('/')[0]
                        sym3_token2 = sym3.split('/')[1]
                        if((sym2_token1 == sym3_token1) and (sym3_token2 == sym1_token2)):
                            combination = {
                                'base':sym1_token2,
                                'intermediate':sym1_token1,
                                'ticker':sym2_token1,
                            }
                            combinations.append(combination)
                
        
    return combinations
        
wx_combinations_usdt = get_crypto_combinations(market_symbols,'USD')


In [20]:
print(f'No. of crypto combinations: {len(wx_combinations_usdt)}')

cominations_df = pd.DataFrame(wx_combinations_usdt)
cominations_df.head()

No. of crypto combinations: 54


,base,intermediate,ticker
0,USD,BTC,ADA
1,USD,BTC,ATOM
2,USD,BTC,BCH
3,USD,BTC,BTG
4,USD,BTC,DAI


# STEP 2: PERFORM TRIANGULAR ARBITRAGE

## Utility method to fetch the current ticker price

In [21]:
def fetch_current_ticker_price(ticker):
    current_ticker_details = exchange.fetch_ticker(ticker)
    ticker_price = current_ticker_details['close'] if current_ticker_details is not None else None
    return ticker_price

## Triangular Arbitrage

In [22]:
def check_buy_buy_sell(scrip1, scrip2, scrip3,initial_investment):
    
    ## SCRIP1
    investment_amount1 = initial_investment
    current_price1 = fetch_current_ticker_price(scrip1)
    final_price = 0
    scrip_prices = {}
    
    if current_price1 is not None:
        buy_quantity1 = round(investment_amount1 / current_price1, 8)
        time.sleep(1)
        ## SCRIP2
        investment_amount2 = buy_quantity1     
        current_price2 = fetch_current_ticker_price(scrip2)
        if current_price2 is not None:
            buy_quantity2 = round(investment_amount2 / current_price2, 8)
            time.sleep(1)
            ## SCRIP3
            investment_amount3 = buy_quantity2     
            current_price3 = fetch_current_ticker_price(scrip3)
            if current_price3 is not None:
                sell_quantity3 = buy_quantity2
                final_price = round(sell_quantity3 * current_price3,3)
                scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
                
    return final_price, scrip_prices

In [23]:
def check_buy_sell_sell(scrip1, scrip2, scrip3, initial_investment):
    ## SCRIP1
    investment_amount1 = initial_investment
    current_price1 = fetch_current_ticker_price(scrip1)
    final_price = 0
    scrip_prices = {}
    if current_price1 is not None:
        buy_quantity1 = round(investment_amount1 / current_price1, 8)
        time.sleep(1)
        ## SCRIP2
        investment_amount2 = buy_quantity1     
        current_price2 = fetch_current_ticker_price(scrip2)
        if current_price2 is not None:
            sell_quantity2 = buy_quantity1
            sell_price2 = round(sell_quantity2 * current_price2,8)
            time.sleep(1)
            ## SCRIP1
            investment_amount3 = sell_price2     
            current_price3 = fetch_current_ticker_price(scrip3)
            if current_price3 is not None:
                sell_quantity3 = sell_price2
                final_price = round(sell_quantity3 * current_price3,3)
                scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
    return final_price,scrip_prices


In [24]:
def check_profit_loss(total_price_after_sell,initial_investment,transaction_brokerage, min_profit):
    apprx_brokerage = transaction_brokerage * initial_investment/100 * 3
    min_profitable_price = initial_investment + apprx_brokerage + min_profit
    profit_loss = round(total_price_after_sell - min_profitable_price,3)
    return profit_loss

# STEP 3: PLACE THE TRADE ORDERS

In [25]:
def place_buy_order(scrip, quantity, limit):
    order = exchange.create_limit_buy_order(scrip, quantity, limit)
    return order

def place_sell_order(scrip, quantity, limit):
    order = exchange.create_limit_sell_order(scrip, quantity, limit)
    return order 

def place_trade_orders(type, scrip1, scrip2, scrip3, initial_amount, scrip_prices):
    final_amount = 0.0
    if type == 'BUY_BUY_SELL':
        s1_quantity = initial_amount/scrip_prices[scrip1]
        place_buy_order(scrip1, s1_quantity, scrip_prices[scrip1])
        
        s2_quantity = s1_quantity/scrip_prices[scrip2]
        place_buy_order(scrip2, s2_quantity, scrip_prices[scrip2])
        
        s3_quantity = s2_quantity
        place_sell_order(scrip3, s3_quantity, scrip_prices[scrip3])
        
    elif type == 'BUY_SELL_SELL':
        s1_quantity = initial_amount/scrip_prices[scrip1]
        place_buy_order(scrip1, s1_quantity, scrip_prices[scrip1])
        
        s2_quantity = s1_quantity
        place_sell_order(scrip2, s2_quantity, scrip_prices[scrip2])
        
        s3_quantity = s2_quantity * scrip_prices[scrip2]
        place_sell_order(scrip3, s3_quantity, scrip_prices[scrip3])
        
        
    return final_amount

Sample order from exchange immediately after execution:   
{'info': {'id': '2490462375', 'symbol': 'btcusdt', 'type': 'limit', 'side': 'buy', 'status': 'wait', 'price': '43201.0', 'origQty': '0.002314', 'executedQty': '0.0', 'createdTime': '1646302254000', 'updatedTime': '1646302254000'}, 'id': '2490462375', 'clientOrderId': None, 'timestamp': 1646302254000, 'datetime': '2022-03-03T10:10:54.000Z', 'lastTradeTimestamp': 1646302254000, 'status': 'open', 'symbol': 'BTC/USDT', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 43201.0, 'amount': None, 'filled': 0.0, 'remaining': None, 'cost': 0.0, 'fee': None, 'average': None, 'trades': [], 'fees': []}

# STEP 4: WRAPPING IT TOGETHER

In [26]:
def perform_triangular_arbitrage(scrip1, scrip2, scrip3, arbitrage_type,initial_investment, 
                               transaction_brokerage, min_profit):
    final_price = 0.0
    if(arbitrage_type == 'BUY_BUY_SELL'):
        # Check this combination for triangular arbitrage: scrip1 - BUY, scrip2 - BUY, scrip3 - SELL
        final_price, scrip_prices = check_buy_buy_sell(scrip1, scrip2, scrip3,initial_investment)
        
    elif(arbitrage_type == 'BUY_SELL_SELL'):
        # Check this combination for triangular arbitrage: scrip1 - BUY, scrip2 - SELL, scrip3 - SELL
        final_price, scrip_prices = check_buy_sell_sell(scrip1, scrip2, scrip3,initial_investment)
        
    profit_loss = check_profit_loss(final_price,initial_investment, transaction_brokerage, min_profit)

    if profit_loss>0:
        print(f"PROFIT-{datetime.now().strftime('%H:%M:%S')}:"\
              f"{arbitrage_type}, {scrip1},{scrip2},{scrip3}, Profit/Loss: {round(final_price-initial_investment,3)} ")
        #place_trade_orders(arbitrage_type, scrip1, scrip2, scrip3, initial_investment, scrip_prices)



In [128]:
INVESTMENT_AMOUNT_DOLLARS = 100
MIN_PROFIT_DOLLARS = 0.5
BROKERAGE_PER_TRANSACTION_PERCENT = 0.2
#while(1):
for combination in wx_combinations_usdt:

    base = combination['base']
    intermediate = combination['intermediate']
    ticker = combination['ticker']


    s1 = f'{intermediate}/{base}'    # Eg: BTC/USDT
    s2 = f'{ticker}/{intermediate}'  # Eg: ETH/BTC
    s3 = f'{ticker}/{base}'          # Eg: ETH/USDT 

    # Check triangular arbitrage for buy-buy-sell 
    perform_triangular_arbitrage(s1,s2,s3,'BUY_BUY_SELL',INVESTMENT_AMOUNT_DOLLARS,
                              BROKERAGE_PER_TRANSACTION_PERCENT, MIN_PROFIT_DOLLARS)
    # Sleep to avoid rate limit on api calls (RateLimitExceeded exception)
    time.sleep(1) 
    # Check triangular arbitrage for buy-sell-sell 
    perform_triangular_arbitrage(s3,s2,s1,'BUY_SELL_SELL',INVESTMENT_AMOUNT_DOLLARS,
                              BROKERAGE_PER_TRANSACTION_PERCENT, MIN_PROFIT_DOLLARS)
    time.sleep(1)    
         
    

PROFIT-14:17:50:BUY_SELL_SELL, ATOM/USD,ATOM/BTC,BTC/USD, Profit/Loss: 4.171 
PROFIT-14:18:07:BUY_SELL_SELL, BTG/USD,BTG/BTC,BTC/USD, Profit/Loss: 1.146 
PROFIT-14:18:39:BUY_SELL_SELL, DOT/USD,DOT/BTC,BTC/USD, Profit/Loss: 3.961 
PROFIT-14:19:15:BUY_BUY_SELL, BTC/USD,GAS/BTC,GAS/USD, Profit/Loss: 4.033 
PROFIT-14:19:44:BUY_SELL_SELL, MNC/USD,MNC/BTC,BTC/USD, Profit/Loss: 113.973 
PROFIT-14:20:24:BUY_SELL_SELL, SHIB/USD,SHIB/BTC,BTC/USD, Profit/Loss: 2363.774 
PROFIT-14:20:32:BUY_SELL_SELL, SMART/USD,SMART/BTC,BTC/USD, Profit/Loss: 12.093 
PROFIT-14:21:25:BUY_BUY_SELL, BTC/USD,XTZ/BTC,XTZ/USD, Profit/Loss: 7.185 
PROFIT-14:22:17:BUY_SELL_SELL, MNC/USD,MNC/ETH,ETH/USD, Profit/Loss: 141.94 
PROFIT-14:23:49:BUY_BUY_SELL, USDT/USD,DOT/USDT,DOT/USD, Profit/Loss: 1.925 
PROFIT-14:24:13:BUY_BUY_SELL, USDT/USD,EXM/USDT,EXM/USD, Profit/Loss: 1.91 


In [129]:
current_ticker_details = exchange.fetch_ticker("Fake")
current_ticker_details

BadSymbol: exmo does not have market symbol Fake

In [27]:
def print_combo_as_3trades(combo):
    base = combo['base']
    intermediate = combo['intermediate']
    ticker = combo['ticker']

    print(f'T1: {intermediate}/{base} \nT2: {ticker}/{intermediate}\nT3: {ticker}/{"USDT"}\n\n')        

## Try and find trades to get from USD -> USDT with smallest loss

In [28]:
exchange = ccxt.exmo({
})
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets]


# function to get all of the 3 step trades starting with .../USD and ending in .../USDT
def get_usd_usdt_combinations(market_symbols, base):
    combinations = []
    for sym1 in market_symbols:
        
        sym1_token1 = sym1.split('/')[0]
        sym1_token2 = sym1.split('/')[1]
        
        if (sym1_token2 == base):
            for sym2 in market_symbols:
                sym2_token1 = sym2.split('/')[0]
                sym2_token2 = sym2.split('/')[1]
                if (sym1_token1 == sym2_token2):
                    for sym3 in market_symbols:
                        sym3_token1 = sym3.split('/')[0]
                        sym3_token2 = sym3.split('/')[1]
                        if((sym2_token1 == sym3_token1) and (sym3_token2 == "USDT")):

                            print(sym1_token1, sym1_token2, sym2_token1, sym2_token2, sym3_token1, sym3_token2)
                            combination = {
                                'base':sym1_token2,
                                'intermediate':sym1_token1,
                                'ticker':sym2_token1,
                            }
                            combinations.append(combination)
                
        
    return combinations
        
usd_to_usdt_combinations = get_usd_usdt_combinations(market_symbols,'USD')
for combo in usd_to_usdt_combinations:
    print_combo_as_3trades(combo)


BTC USD ADA BTC ADA USDT
BTC USD ALGO BTC ALGO USDT
BTC USD BCH BTC BCH USDT
BTC USD CRON BTC CRON USDT
BTC USD DASH BTC DASH USDT
BTC USD DOT BTC DOT USDT
BTC USD ETC BTC ETC USDT
BTC USD ETH BTC ETH USDT
BTC USD EXM BTC EXM USDT
BTC USD GMT BTC GMT USDT
BTC USD IQN BTC IQN USDT
BTC USD NEAR BTC NEAR USDT
BTC USD PRQ BTC PRQ USDT
BTC USD ROOBEE BTC ROOBEE USDT
BTC USD SHIB BTC SHIB USDT
BTC USD SOL BTC SOL USDT
BTC USD TON BTC TON USDT
BTC USD USDC BTC USDC USDT
BTC USD WXT BTC WXT USDT
BTC USD XRP BTC XRP USDT
ETH USD BCH ETH BCH USDT
ETH USD CRON ETH CRON USDT
ETH USD EXM ETH EXM USDT
ETH USD USDC ETH USDC USDT
ETH USD XRP ETH XRP USDT
LTC USD ETH LTC ETH USDT
USDT USD BTC USDT BTC USDT
USDT USD ADA USDT ADA USDT
USDT USD ALGO USDT ALGO USDT
USDT USD BCH USDT BCH USDT
USDT USD CRON USDT CRON USDT
USDT USD DASH USDT DASH USDT
USDT USD DOT USDT DOT USDT
USDT USD ETC USDT ETC USDT
USDT USD ETH USDT ETH USDT
USDT USD EXM USDT EXM USDT
USDT USD GMT USDT GMT USDT
USDT USD IQN USDT IQN USD

In [42]:
def check_buy_buy_sell2(pair1, pair2, pair3, initial_investment):
    
    ## TRADE 1
    investment_amount1 = initial_investment
    current_price1 = fetch_current_ticker_price(pair1)

    # will be denominated in 
    final_quantity = 0
    intermediate_trade_prices = {}
    
    if current_price1 is not None:
        buy_quantity1 = round(investment_amount1 / current_price1, 8)
        time.sleep(1)
        
        ## TRADE 2
        investment_amount2 = buy_quantity1     
        current_price2 = fetch_current_ticker_price(pair2)
        if current_price2 is not None:
            buy_quantity2 = round(investment_amount2 / current_price2, 8)
            time.sleep(1)

            ## TRADE 3
            investment_amount3 = buy_quantity2     
            current_price3 = fetch_current_ticker_price(pair3)
            if current_price3 is not None:
                sell_quantity3 = buy_quantity2

                # sell output of TRADE 2 at the price of PAIR 3
                final_quantity = round(sell_quantity3 * current_price3, 3)
                # token1_pair3 = pair3.split('/')[0]
                token2_pair3 = pair3.split('/')[1]
             
                final_price_usd = fetch_current_ticker_price(f"{token2_pair3}/USD") * final_quantity
                intermediate_trade_prices = {pair1 : current_price1, pair2 : current_price2, pair3 : current_price3}
                
    return final_quantity, final_price_usd ,intermediate_trade_prices


INITIAL_MONEY=100
OUTPUT_COIN = "USDT"
for combo in usd_to_usdt_combinations:
    base = combo['base']
    intermediate = combo['intermediate']
    ticker = combo['ticker']

    trade1 = f'{intermediate}/{base}'    
    trade2 = f'{ticker}/{intermediate}' 
    trade3 = f'{ticker}/{OUTPUT_COIN}'   
          
    final_quantity, final_price_usd, intermediate_prices = check_buy_buy_sell2(trade1, trade2, trade3, INITIAL_MONEY)
    
    if final_quantity > 90:
        print_combo_as_3trades(combo)
        print("final price: ", final_quantity, "final price usd: ", final_price_usd, "intermediate prices: ", intermediate_prices,)

    
    

T1: BTC/USD 
T2: ADA/BTC
T3: ADA/USDT


final price:  90.261 final price usd:  100.370232 intermediate prices:  {'BTC/USD': 32187.36, 'ADA/BTC': 1.763e-05, 'ADA/USDT': 0.51219574}
T1: BTC/USD 
T2: DOT/BTC
T3: DOT/USDT


final price:  90.074 final price usd:  100.162288 intermediate prices:  {'BTC/USD': 32177.56, 'DOT/BTC': 0.00032969, 'DOT/USDT': 9.5556}
T1: BTC/USD 
T2: ETC/BTC
T3: ETC/USDT


final price:  90.343 final price usd:  100.46141600000001 intermediate prices:  {'BTC/USD': 32177.56, 'ETC/BTC': 0.00068799, 'ETC/USDT': 20.0}
T1: BTC/USD 
T2: PRQ/BTC
T3: PRQ/USDT


final price:  97.73 final price usd:  108.67576000000001 intermediate prices:  {'BTC/USD': 32207.98, 'PRQ/BTC': 4.12e-06, 'PRQ/USDT': 0.12968415}
T1: BTC/USD 
T2: ROOBEE/BTC
T3: ROOBEE/USDT


final price:  101.111 final price usd:  112.43543200000002 intermediate prices:  {'BTC/USD': 32207.98, 'ROOBEE/BTC': 5e-08, 'ROOBEE/USDT': 0.00162829}
T1: BTC/USD 
T2: WXT/BTC
T3: WXT/USDT


final price:  99.365 final price usd: